In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("mapReduce").master("spark://spark-master:7077").getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 13:21:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.7/dist-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [3]:
sc.parallelize([1,2,3,4,5,6,7,8,9,0]).map(lambda x: x **2).sum() # Here We are creating an RDD and we're performing a calculation on its elements (Square) and getting the sum

285

In [4]:
word_list = ["cat","rat","dog","bird","lizard","chicken","cat","rat"]
wordsRDD = sc.parallelize(word_list,3)
print(type(wordsRDD))
wordsRDD.collect()

<class 'pyspark.rdd.RDD'>


['cat', 'rat', 'dog', 'bird', 'lizard', 'chicken', 'cat', 'rat']

#### RDDs support two types of operations: transformations, which create a new dataset from an existing one, and actions, which return a value to the driver program after running a computation on the dataset. For example, map is a transformation that passes each dataset element through a function and returns a new RDD representing the results. On the other hand, reduce is an action that aggregates all the elements of the RDD using some function and returns the final result to the driver program (although there is also a parallel reduceByKey that returns a distributed dataset).

In [5]:
def make_plural(w):    # Create a function that adds an s to the element that it is passed
    return w + "s"

print(make_plural("cat"))   # Use of the function
# Here we're gonna TRANSFORM our wordsRDD into a new RDD
plural_wordRDD = wordsRDD.map(make_plural)
print(plural_wordRDD.first()) # Here the first is anaction because it returns a value 
print(plural_wordRDD.take(2))
print(plural_wordRDD.collect())

cats
cats
['cats', 'rats']
['cats', 'rats', 'dogs', 'birds', 'lizards', 'chickens', 'cats', 'rats']


## Key Value Pairs

In [6]:
wordPairs = wordsRDD.map(lambda d:(d,1))
print(wordPairs.collect())

[('cat', 1), ('rat', 1), ('dog', 1), ('bird', 1), ('lizard', 1), ('chicken', 1), ('cat', 1), ('rat', 1)]


## Word Count

In [23]:
word_list = ["cat","rat","dog","horse","cat","rat"]
wordsRDD = sc.parallelize(word_list,3)
wordsCountCollected = (wordsRDD.map(lambda x :(x,1)).reduceByKey(lambda a,b : a+b).collect() )
print(wordsCountCollected)

# .map() = Creating a (key,value) pair where the keys are the words and the values are 1 
# .reduceByKey() = Apply the redudeByKey func where all values with the same key are grouped and a reduction operation is performed
# .collect() = The output value is retrieved

[('cat', 2), ('horse', 1), ('rat', 2), ('dog', 1)]


# Explanation on the reduceByKey

This function operates on a key-value pair RDD (Resilient Distributed Dataset), where each element in the RDD is a pair (key, value). The main purpose of reduceByKey() is to perform a reduction operation on elements with the same key. It combines values for each key using a specified function and returns an RDD of key-value pairs where each key is unique.

Here's how it works:

1. Grouping by Key: First, it groups together all the values that have the same key. This step is done in parallel across the Spark cluster.

2. Applying the Reduction Operation: Then, for each group of values with the same key, it applies a specified function (usually a commutative and associative function) to reduce those values to a single value.

3. Output: Finally, it returns an RDD where each unique key is associated with a single value, which is the result of the reduction operation applied to the values with that key.

In [16]:
# In this example We-re gonna Concatenate on a list

data  = [
    ("fruit",["apple","bananna","orange"]),
    ("color",["red","green","blue"]),
    ("fruit",["pineapple","tangerine"]),
    ("color",["black"]),
    ("color",["orange","yellow"]),
    ("fruit",["mango"])
]
rdd_test = sc.parallelize(data,4)
concatenated_rdd = rdd_test.reduceByKey(lambda x,y : x+y).collect()
print(concatenated_rdd)

[('color', ['orange', 'yellow', 'black', 'red', 'green', 'blue']), ('fruit', ['pineapple', 'tangerine', 'apple', 'bananna', 'orange', 'mango'])]


# Using cache

As we know spark is lazy which means that operations are recorder but only executed when actions are performed such as collect() and count() and so on. it's important to remember that operations return an atual item to the program being a number .count() or a list with .collect().

Becase the fact that operations run from the start, they may be inefficient and in many cases we might need to cache the result the first time an operation is run on and RDD so it can be referenced by other operations without having to execute the whole of the code again.

In [17]:
#Create an RDD
random_words = ["letter","box","magician","fan","desktop"]
random_wordsRDD = sc.parallelize(random_words)
print(random_wordsRDD)
print(random_wordsRDD.count())

ParallelCollectionRDD[43] at readRDDFromFile at PythonRDD.scala:289
5


In [18]:
# When I execute this code, it will be ran from the start
random_wordsRDD.count()

5

In [19]:
#let's cache the 
random_wordsRDD.cache() 
# We need to rerun again from the start but know that we have give the instruction to cache. it will store the result
random_wordsRDD.count()

5

In [22]:
# When running this again : random_wordsRDD.count() , the "sc.parallelize(random_words)" expression won't be run again
random_wordsRDD.count()
#  Where is this useful: it is when you have branching parts or loops, so that you dont do things again and again. 
#  Spark, being "lazy" will rerun the chain again. So cache or persist serves as a checkpoint, breaking the RDD chain or the lineage.

5

<hr>

In [27]:

#Using SparkRDD operations and python in tandem

#Python 
birds_list = ["heron","owl","eagle"]
animals_list = birds_list + word_list

animals_dict = {}
for i in word_list:
    animals_dict[i] = "mammal"
for i in birds_list:
    animals_dict[i] = "bird"
print(animals_dict)

#SparkRDD Operations
animsRDD = sc.parallelize(animals_list,4)
animsRDD.cache()
mammals_count = animsRDD.filter(lambda x : animals_dict[x] == "mammal").count()
birds_count = animsRDD.filter(lambda y: animals_dict[y] =="bird").count()
print(mammals_count,birds_count)


{'cat': 'mammal', 'rat': 'mammal', 'dog': 'mammal', 'horse': 'mammal', 'heron': 'bird', 'owl': 'bird', 'eagle': 'bird'}
6 3


In [28]:
spark.stop()